In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [185]:
data = pd.read_csv('./data/fu2405_tick.csv')

data.tail()

,date,volume,turnover,open_interest,last_price,bid_price,ask_price,bid_volume,ask_volume
745468,2024-04-30 14:59:47,10,376800.0,4036.0,3800.0,0.0,4000.0,0,3
745469,2024-04-30 14:59:47,10,376800.0,4036.0,3800.0,3614.0,4000.0,1,3
745470,2024-04-30 14:59:48,10,376800.0,4036.0,3800.0,3614.0,4000.0,1,3
745471,2024-04-30 14:59:50,10,376800.0,4036.0,3800.0,0.0,4000.0,0,3
745472,2024-04-30 15:00:00,10,376800.0,4036.0,3800.0,0.0,4000.0,0,3


In [191]:
mul = 10

def cal_trade(x):
    if x[2] == 0.0:
        return x[0]
    else:
        return  x[1]/x[2]/mul

data['volume_delta'] = data['volume'].diff()
data['turn_delta'] = data['turnover'].diff()
data['trade_price'] = data[['last_price','turn_delta','volume_delta']].apply(cal_trade,axis=1)

In [192]:
data.head()

,date,volume,turnover,open_interest,last_price,bid_price,ask_price,bid_volume,ask_volume,volume_delta,turn_delta,trade_price
0,2024-03-21 10:43:41,381296,1.279356e+10,237099.0,3371.0,3371.0,3372.0,185,131,NaN,NaN,NaN
1,2024-03-21 10:43:41,381313,1.279414e+10,237083.0,3372.0,3371.0,3372.0,184,115,17.0,573230.0,3371.941176
2,2024-03-21 10:43:42,381314,1.279417e+10,237082.0,3371.0,3371.0,3372.0,185,115,1.0,33710.0,3371.000000
3,2024-03-21 10:43:42,381316,1.279424e+10,237080.0,3372.0,3371.0,3372.0,184,113,2.0,67440.0,3372.000000
4,2024-03-21 10:43:43,381316,1.279424e+10,237080.0,3372.0,3371.0,3372.0,188,121,0.0,0.0,3372.000000


In [193]:
def argmax(x):
    return x.argmax()
def argmin(x):
    return x.argmin()
def get_diff(x):
    if abs(x[0])>abs(x[1]):
        return x[0]
    else:
        return x[1]

In [194]:
p0 = 60
p1 = 240
p2 = 360


roll_0 = data['last_price'].rolling(p0)
roll_1 = data['last_price'].rolling(p1)
roll_2 = data['last_price'].rolling(p2)


max0 = roll_0.max()
min0 = roll_0.min()

max1 = roll_1.max()
min1 = roll_1.min()

max2 = roll_2.max().shift(-p2)
min2 = roll_2.min().shift(-p2)


data['min0_diff'] = data['last_price'] - max0
data['max0_diff'] = data['last_price'] - min0
data['min1_diff'] = data['last_price'] - max1 
data['max1_diff'] = data['last_price'] - min1
data['max2_diff'] = max2 - data['last_price']
data['min2_diff'] = min2 - data['last_price']

data['diff0'] = data[['min0_diff','max0_diff']].apply(get_diff,axis=1)
data['diff1']= data[['min1_diff','max1_diff']].apply(get_diff,axis=1)
data['diff2']=data[['min2_diff','max2_diff']].apply(get_diff,axis=1)

In [195]:
dt = data[:-20000]
dt['diff2'].describe()


count    725473.000000
mean          0.372578
std           7.254364
min         -98.000000
25%          -3.000000
50%           2.000000
75%           4.000000
max          54.000000
Name: diff2, dtype: float64

In [284]:
dt[(dt['last_price']<=dt['bid_price'])&(dt['bid_volume']>2*dt['ask_volume'])&(dt['volume_delta']>=10)]['max2_diff'].describe()

count    7647.000000
mean        4.645220
std         5.099051
min        -1.000000
25%         2.000000
50%         3.000000
75%         5.000000
max        53.000000
Name: max2_diff, dtype: float64

In [285]:
dt[(dt['last_price']>=dt['ask_price'])&(dt['ask_volume']>4 *dt['bid_volume'])&(dt['volume_delta']<=10)]['min2_diff'].describe()

count    10873.000000
mean        -4.515589
std          6.255694
min        -96.000000
25%         -5.000000
50%         -3.000000
75%         -2.000000
max         28.000000
Name: min2_diff, dtype: float64